In [4]:
import cvxpy as cp
import numpy as np
import math


In [5]:
A = np.array([[ 1, 1, 0, 0, 0],
[ 0, -1, 1, 1, 0],
[ 0, 0, 0, -1, 1],
[-1, 0, -1, 0, -1]], dtype=np.float64)
A = A[0:-1,:]
R1 = 1.
R2 = 1e3
R5 = 1e2
VT = 26
IS = 1
VS = 1e3


In [6]:
ij = cp.Variable(5)
OBJ1 = VS*ij[0] + (1./2)*R1*cp.square(ij[0])
OBJ2 = (1./2)*R2*cp.square(ij[1])
OBJ3 = VT*IS*(-cp.entr(1. + ij[2]/IS) - ij[2]/IS)
OBJ4 = VT*IS*(-cp.entr(1. + ij[3]/IS) - ij[3]/IS)
OBJ5 = (1./2)*R5*cp.square(ij[4])
obj = cp.Minimize(OBJ1 + OBJ2 + OBJ3 + OBJ4 + OBJ5)
constr = [A * ij == 0.]
problem = cp.Problem(obj, constr)
#problem.solve(verbose=True, solver=SCS, eps=1e-4)
problem.solve(verbose=True)


                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) May 13 02:18:03 AM: Your problem has 5 variables, 1 constraints, and 0 parameters.
(CVXPY) May 13 02:18:03 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 13 02:18:03 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 13 02:18:03 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 13 02:18:03 AM: Compiling problem (target solver=ECOS).
(CVXPY) May 13 02:18:03 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

c:\Users\Latitude 5289 2-in-1\AppData\Local\Programs\Python\Python310\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


-490.77819799933167

In [8]:
e = -problem.constraints[0].dual_value
v = np.array([[VS + R1*float(ij.value[0])],
[R2*float(ij.value[1])],
[VT*math.log(1. + float(ij.value[2])/IS)],
[VT*math.log(1. + float(ij.value[3])/IS)],
[R5*float(ij.value[4])]])
v_err = v - np.transpose(A) * e
rel_err = np.linalg.norm(v_err) / np.linalg.norm(v)
print("Relative error in voltage: %e\n" % rel_err)
print(v)
print(ij.value)

Relative error in voltage: 1.420047e+00

[[999.01703386]
 [982.96613594]
 [ 16.05094564]
 [  3.15391948]
 [ 12.89687387]]
[-0.98296614  0.98296614  0.8539974   0.12896874  0.12896874]
